In [17]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.nn.init import xavier_normal_, xavier_uniform_
import argparse
import numpy as np
import time
import sys
import pandas as pd
from os.path import abspath

# from base.graph_recommender import GraphRecommender
from util.sampler import next_batch_pairwise_kg, next_batch_pairwise
from util.conf import OptionConf
import torch
import torch.nn as nn 
import torch.nn.functional as F
from scipy.sparse import coo_matrix
from util.loss_torch import bpr_loss, l2_reg_loss, EmbLoss, contrastLoss, l2_reg_loss
from util.init import *
from base.torch_interface import TorchGraphInterface
import os
import numpy as np 
import time 
from torch.optim.lr_scheduler import ReduceLROnPlateau
from itertools import product

from data.loader import FileIO
from util.conf import ModelConf
from base.recommender import Recommender
from data.ui_graph import Interaction
# from data.knowledge import Knowledge
from util.algorithm import find_k_largest
from time import strftime, localtime
from data.loader import FileIO
from util.evaluation import ranking_evaluation

In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
device

device(type='cuda')

## Base Recommender

In [46]:
class GraphRecommender(Recommender):
    def __init__(self, conf, training_set, test_set, knowledge_set, **kwargs):
        super(GraphRecommender, self).__init__(conf, training_set, test_set, knowledge_set, **kwargs)
        self.data = Interaction(conf, training_set, test_set, knowledge_set, self.knowledge)
        self.bestPerformance = []
        top = self.ranking['-topN'].split(',')
        self.topN = [int(num) for num in top]
        self.max_N = max(self.topN)
        
        self.output = f"./results/{self.model_name}/{self.dataset}/@{self.model_name}-inp_emb:{kwargs['input_dim']}-hyper_emb:{kwargs['hyper_dim']}-bs:{self.batch_size}-lr:{kwargs['lr']}-lrd:{kwargs['lr_decay']}-reg:{kwargs['reg']}-leaky:{kwargs['p']}-dropout:{kwargs['drop_rate']}-n_layers:{kwargs['n_layers']}-n_heads:{kwargs['n_heads']}-n_self_att:{kwargs['n_self_att']}/"
        if not os.path.exists(self.output):
            os.makedirs(self.output)

    def print_model_info(self):
        super(GraphRecommender, self).print_model_info()
        # # print dataset statistics
        print('Training Set Size: (user number: %d, item number %d, interaction number: %d)' % (self.data.training_size()))
        print('Test Set Size: (user number: %d, item number %d, interaction number: %d)' % (self.data.test_size()))
        print('=' * 80)

    def build(self):
        pass

    def train(self):
        pass

    def predict(self, u):
        pass

    def test(self, user_emb, item_emb):
        def process_bar(num, total):
            rate = float(num) / total
            ratenum = int(50 * rate)
            r = '\rProgress: [{}{}]{}%'.format('+' * ratenum, ' ' * (50 - ratenum), ratenum*2)
            sys.stdout.write(r)
            sys.stdout.flush()

        # predict
        rec_list = {}
        user_count = len(self.data.test_set)
        for i, user in enumerate(self.data.test_set):
            # s_find_candidates = time.time()
            
            # candidates = predict(user)
            user_id  = self.data.get_user_id(user)
            score = torch.matmul(user_emb[user_id], item_emb.transpose(0, 1))
            candidates = score.cpu().numpy()
            
            # e_find_candidates = time.time()
            # print("Calculate candidates time: %f s" % (e_find_candidates - s_find_candidates))
            # predictedItems = denormalize(predictedItems, self.data.rScale[-1], self.data.rScale[0])
            rated_list, li = self.data.user_rated(user)
            for item in rated_list:
                candidates[self.data.item[item]] = -10e8
            
            # s_find_k_largest = time.time()
            ids, scores = find_k_largest(self.max_N, candidates)
            # e_find_k_largest = time.time()
            # print("Find k largest candidates: %f s" % (e_find_k_largest - s_find_k_largest))
            item_names = [self.data.id2item[iid] for iid in ids]
            rec_list[user] = list(zip(item_names, scores))
            if i % 1000 == 0:
                process_bar(i, user_count)
        process_bar(user_count, user_count)
        print('')
        return rec_list

    def evaluate(self, rec_list):
        self.recOutput.append('userId: recommendations in (itemId, ranking score) pairs, * means the item is hit.\n')
        for user in self.data.test_set:
            line = str(user) + ':'
            for item in rec_list[user]:
                line += ' (' + str(item[0]) + ',' + str(item[1]) + ')'
                if item[0] in self.data.test_set[user]:
                    line += '*'
            line += '\n'
            self.recOutput.append(line)
        current_time = strftime("%Y-%m-%d %H-%M-%S", localtime(time.time()))
        # output prediction result
        out_dir = self.output
        # out_dir = self.output['-dir']
        file_name = self.config['model.name'] + '@' + current_time + '-top-' + str(self.max_N) + 'items' + '.txt'
        FileIO.write_file(out_dir, file_name, self.recOutput)
        print('The result has been output to ', abspath(out_dir), '.')
        file_name = self.config['model.name'] + '@' + current_time + '-performance' + '.txt'
        self.result = ranking_evaluation(self.data.test_set, rec_list, self.topN)
        self.model_log.add('###Evaluation Results###')
        self.model_log.add(self.result)
        FileIO.write_file(out_dir, file_name, self.result)
        print('The result of %s:\n%s' % (self.model_name, ''.join(self.result)))

    def fast_evaluation(self, model, epoch, user_emb, item_emb, kwargs=None):
        print('Evaluating the model...')
        s_test = time.time()
        rec_list = self.test(user_emb, item_emb)
        e_test = time.time() 
        print("Test time: %f s" % (e_test - s_test))
        
        s_measure = time.time()
        all_measures = ranking_evaluation(self.data.test_set, rec_list, self.topN)
        len_measures = len(self.topN)
        lst_res = []
    
        data_ep = {}
        data_ep['epoch'] = epoch
        for i in range(0, len(all_measures), 5):
            mes = all_measures[i:i+5]
            topk = int(mes[0].split(' ')[1][:-1])
            hitk = float(mes[1].split(':')[1][:-1])
            preck = float(mes[2].split(':')[1][:-1])
            reck = float(mes[3].split(':')[1][:-1])
            ndcgk = float(mes[4].split(':')[1][:-1])
            data_ep[f'hit@{topk}'] = hitk
            data_ep[f'precision@{topk}'] = preck
            data_ep[f'recall@{topk}'] = reck
            data_ep[f'ndcg@{topk}'] = ndcgk

        measure = all_measures[(len_measures - 1) * 5: (len_measures - 1) * 5 + 5]
        e_measure = time.time()
        print("Measure time: %f s" % (e_measure - s_measure))
        if len(self.bestPerformance) > 0:
            count = 0
            performance = {}
            for m in measure[1:]:
                k, v = m.strip().split(':')
                performance[k] = float(v)
            for k in self.bestPerformance[1]:
                if self.bestPerformance[1][k] > performance[k]:
                    count += 1
                else:
                    count -= 1
            if count < 0:
                self.bestPerformance[1] = performance
                self.bestPerformance[0] = epoch + 1
                self.save(model)
        else:
            self.bestPerformance.append(epoch + 1)
            performance = {}
            for m in measure[1:]:
                k, v = m.strip().split(':')
                performance[k] = float(v)
            self.bestPerformance.append(performance)
            self.save(model)

        print('-' * 120)
        print('Real-Time Ranking Performance ' + ' (Top-' + str(self.max_N) + ' Item Recommendation)')
        measure = [m.strip() for m in measure[1:]]
        print('*Current Performance*')
        print('Epoch:', str(epoch + 1) + ',', '  |  '.join(measure))
        bp = ''
        # for k in self.bestPerformance[1]:
        #     bp+=k+':'+str(self.bestPerformance[1][k])+' | '
        bp += 'Hit Ratio' + ':' + str(self.bestPerformance[1]['Hit Ratio']) + '  |  '
        bp += 'Precision' + ':' + str(self.bestPerformance[1]['Precision']) + '  |  '
        bp += 'Recall' + ':' + str(self.bestPerformance[1]['Recall']) + '  |  '
        # bp += 'F1' + ':' + str(self.bestPerformance[1]['F1']) + ' | '
        bp += 'NDCG' + ':' + str(self.bestPerformance[1]['NDCG'])
        print('*Best Performance* ')
        print('Epoch:', str(self.bestPerformance[0]) + ',', bp)
        print('-' * 120)
        return measure, data_ep
    
    def save(self, model):
        with torch.no_grad():
            self.best_user_emb, self.best_item_emb, _ = model()
        self.save_model(model)
    
    def save_model(self, model):
        # save model 
        current_time = strftime("%Y-%m-%d", localtime(time.time()))
        out_dir = self.output
        file_name =  self.config['model.name'] + '@' + current_time + '-weight' + '.pth'
        weight_file = out_dir + '/' + file_name 
        torch.save(model.state_dict(), weight_file)

    def save_loss(self, train_losses, rec_losses, reg_losses):
        df_train_loss = pd.DataFrame(train_losses, columns = ['ep', 'loss'])
        df_rec_loss = pd.DataFrame(rec_losses, columns = ['ep', 'loss'])
        df_reg_loss = pd.DataFrame(reg_losses, columns = ['ep', 'loss'])
        df_train_loss.to_csv(self.output + '/train_loss.csv')
        df_rec_loss.to_csv(self.output + '/rec_loss.csv')
        df_reg_loss.to_csv(self.output + '/reg_loss.csv')

    def save_perfomance_training(self, log_train):
        df_train_log = pd.DataFrame(log_train)
        df_train_log.to_csv(self.output + '/train_performance.csv')

## Layers

In [4]:
class HGNNConv(nn.Module):
    def __init__(self, leaky, hyper_dim, bias=False):
        super(HGNNConv, self).__init__()
        self.hyper_dim = hyper_dim
        self.act = nn.LeakyReLU(negative_slope=leaky)
        self.fc1 = nn.Linear(hyper_dim, hyper_dim ,bias=False) 
        self.fc2 = nn.Linear(hyper_dim, hyper_dim ,bias=False)  
        self.fc3 = nn.Linear(hyper_dim, hyper_dim ,bias=False)  
        
        self.bn = torch.nn.BatchNorm1d(hyper_dim)
        if bias:
            self.bias = nn.Parameter(torch.FloatTensor(hyper_dim))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()
        
    def reset_parameters(self):
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, adj, embeds):
        lat1 = self.act(adj.T @ embeds)
        lat2 = self.act(self.fc1(lat1)) +  lat1
        lat3 = self.act(self.fc2(lat2)) + lat2
        lat4 = self.act(self.fc3(lat3)) + lat3 
        output = adj @ lat4
        if self.bias is not None:
            output += self.bias 
        output = self.bn(output)
        ret = self.act(output)
        return ret

## Model

In [11]:
class Model(nn.Module):
    def __init__(self, config, data, args):
        super(Model, self).__init__()
        self.data = data
        adj = data.interaction_mat
        kg_adj = data.kg_interaction_mat
        self.adj  = TorchGraphInterface.convert_sparse_mat_to_tensor(adj).to_dense().to(device)
        self.kg_adj = TorchGraphInterface.convert_sparse_mat_to_tensor(kg_adj).to_dense().to(device)
        
        self._parse_args(args)
        self.embedding_dict = self._init_model()
        
        self.fc_u = nn.Linear(self.input_dim, self.hyper_dim)
        self.fc_i = nn.Linear(self.input_dim, self.hyper_dim)
        self.fc_e = nn.Linear(self.input_dim, self.hyper_dim)
        
        self.hgnn_u = HGNNConv(leaky=self.p, hyper_dim=self.hyper_dim)
        self.hgnn_i = HGNNConv(leaky=self.p, hyper_dim=self.hyper_dim)
        self.hgnn_e = HGNNConv(leaky=self.p, hyper_dim=self.hyper_dim)
        
        self.non_linear = nn.ReLU()
        self.dropout = nn.Dropout(self.drop_rate)
        
    def _parse_args(self, args):
        self.input_dim = args['input_dim']
        self.hyper_dim = args['hyper_dim']
        self.p = args['p']
        self.drop_rate = args['drop_rate'] 
        self.layers = args['n_layers']
    
    def _init_model(self):
        initializer = nn.init.xavier_uniform_
        embedding_dict = nn.ParameterDict({
            'user_emb': nn.Parameter(initializer(torch.empty(self.data.user_num, self.input_dim)).to(device)),
            'item_emb': nn.Parameter(initializer(torch.empty(self.data.item_num, self.input_dim)).to(device)),
            'entity_emb': nn.Parameter(initializer(torch.empty(self.data.entity_num, self.input_dim)).to(device))
        })
        return embedding_dict

    def forward(self):
        uEmbed = self.embedding_dict['user_emb']
        iEmbed = self.embedding_dict['item_emb']
        eEmbed = self.embedding_dict['entity_emb']
        
        uEmbed = self.dropout(self.non_linear(self.fc_u(uEmbed)))
        iEmbed = self.dropout(self.non_linear(self.fc_i(iEmbed)))
        eEmbed = self.dropout(self.non_linear(self.fc_e(eEmbed)))
        
        embeds = torch.cat([uEmbed, iEmbed, eEmbed], 0)
        all_embeddings = [embeds]
        
        for k in range(self.layers):
            # ego_embeddings = torch.sparse.mm(self.sparse_norm_adj, ego_embeddings)
            hyperULat = self.hgnn_u(self.adj, uEmbed)
            hyperILat1 = self.hgnn_i(self.adj.T, iEmbed)
            hyperILat2 = self.hgnn_e(self.kg_adj, iEmbed)
            hyperILat = sum([hyperILat1, hyperILat2])
            
            hyperELat = self.hgnn_e(self.kg_adj.T, eEmbed)

            ego_embeddings = torch.cat([hyperULat, hyperILat, hyperELat], dim=0)
            all_embeddings += [ego_embeddings]
            
        all_embeddings = torch.stack(all_embeddings, dim=1)
        all_embeddings = torch.mean(all_embeddings, dim=1)
        user_all_embeddings = all_embeddings[:self.data.user_num]
        item_all_embeddings = all_embeddings[self.data.user_num:self.data.user_num+ self.data.item_num]
        entity_all_embeddings = all_embeddings[self.data.user_num+ self.data.item_num:]
        
        return user_all_embeddings, item_all_embeddings, entity_all_embeddings


In [1]:
# https://github.com/daiquocnguyen/Graph-Transformer/blob/master/UGformerV2_PyTorch/UGformerV2.py
from torch.nn import TransformerEncoder, TransformerEncoderLayer

class  HGTransformerModel(nn.Module):
    def __init__(self, config, data, args):
        super(Model, self).__init__()
        self.data = data
        adj = data.interaction_mat
        kg_adj = data.kg_interaction_mat
        self.adj  = TorchGraphInterface.convert_sparse_mat_to_tensor(adj).to_dense().to(device)
        self.kg_adj = TorchGraphInterface.convert_sparse_mat_to_tensor(kg_adj).to_dense().to(device)
        
        self._parse_args(args)
        self.embedding_dict = self._init_model()
        
        self.non_linear = nn.ReLU()
        self.dropout = nn.Dropout(self.drop_rate)
        
        
        self.lst_gnn_u = torch.nn.ModuleList()
        self.lst_gnn_i = torch.nn.ModuleList()
        self.lst_gnn_i2 = torch.nn.ModuleList()
        self.lst_gnn_e = torch.nn.ModuleList()
        
        self.ugformer_layers_u = torch.nn.ModuleList()
        self.ugformer_layers_i = torch.nn.ModuleList()
        self.ugformer_layers_e = torch.nn.ModuleList()
        
        for _layer in range(self.layers):
            encoder_layers_u = TransformerEncoderLayer(d_model=self.input_dim, nhead=self.nhead, dim_feedforward=self.hyper_dim, dropout=self.drop_rate)
            encoder_layers_i = TransformerEncoderLayer(d_model=self.input_dim, nhead=self.nhead, dim_feedforward=self.hyper_dim, dropout=self.drop_rate)
            encoder_layers_e = TransformerEncoderLayer(d_model=self.input_dim, nhead=self.nhead, dim_feedforward=self.hyper_dim, dropout=self.drop_rate)
            
            self.ugformer_layers_u.append(TransformerEncoder(encoder_layers_u, self.num_self_att_layers))
            self.ugformer_layers_i.append(TransformerEncoder(encoder_layers_i, self.num_self_att_layers))
            self.ugformer_layers_e.append(TransformerEncoder(encoder_layers_e, self.num_self_att_layers))
            
            self.lst_gnn_u.append( HGNNConv(leaky=self.p, hyper_dim=self.hyper_dim))
            self.lst_gnn_i.append( HGNNConv(leaky=self.p, hyper_dim=self.hyper_dim))
            self.lst_gnn_i2.append( HGNNConv(leaky=self.p, hyper_dim=self.hyper_dim))
            self.lst_gnn_e.append( HGNNConv(leaky=self.p, hyper_dim=self.hyper_dim))
        
    def _parse_args(self, args):
        self.input_dim = args['input_dim']
        self.hyper_dim = args['hyper_dim']
        self.p = args['p']
        self.drop_rate = args['drop_rate'] 
        self.layers = args['n_layers']
        self.nhead = args['n_heads']
        self.num_self_att_layers = args['n_self_att']
        
    def _init_model(self):
        initializer = nn.init.xavier_uniform_
        embedding_dict = nn.ParameterDict({
            'user_emb': nn.Parameter(initializer(torch.empty(self.data.user_num, self.input_dim)).to(device)),
            'item_emb': nn.Parameter(initializer(torch.empty(self.data.item_num, self.input_dim)).to(device)),
            'entity_emb': nn.Parameter(initializer(torch.empty(self.data.entity_num, self.input_dim)).to(device))
        })
        return embedding_dict

    def forward(self):
        uEmbed = self.embedding_dict['user_emb']
        iEmbed = self.embedding_dict['item_emb']
        eEmbed = self.embedding_dict['entity_emb']
        
        for layer_idx in range(self.layers):
            uEmbedTr = torch.unsqueeze(uEmbed, 1) # [node, batch_size=1, dim]
            eEmbedTr = torch.unsqueeze(eEmbed, 1) # [node, batch_size=1, dim]
            iEmbedTr = torch.unsqueeze(iEmbed, 1) # [node, batch_size=1, dim]

            uEmbedTr_ = self.ugformer_layers_u[layer_idx](uEmbedTr)
            iEmbedTr_ = self.ugformer_layers_i[layer_idx](iEmbedTr)
            eEmbedTr_ = self.ugformer_layers_e[layer_idx](eEmbedTr)
            
            uEmbedTr_ = torch.squeeze(uEmbedTr_, 1)
            iEmbedTr_ = torch.squeeze(iEmbedTr_, 1)
            eEmbedTr_ = torch.squeeze(eEmbedTr_, 1)
            
            hyperULat = self.lst_gnn_u(self.adj, uEmbedTr_)
            hyperILat1 = self.lst_gnn_i(self.adj.T, iEmbedTr_)
            hyperILat2 = self.lst_gnn_i2(self.kg_adj, iEmbedTr_)
            hyperELat = self.lst_gnn_e(self.kg_adj.T, eEmbedTr_)
            hyperILat = sum([hyperILat1, hyperILat2])
            
            # cross attention 
            
            
            ego_embeddings = torch.cat([hyperULat, hyperILat, hyperELat], dim=0)
            all_embeddings += [ego_embeddings]
            
        all_embeddings = torch.stack(all_embeddings, dim=1)
        all_embeddings = torch.mean(all_embeddings, dim=1)
        user_all_embeddings = all_embeddings[:self.data.user_num]
        item_all_embeddings = all_embeddings[self.data.user_num:self.data.user_num+ self.data.item_num]
        entity_all_embeddings = all_embeddings[self.data.user_num+ self.data.item_num:]
        
        return user_all_embeddings, item_all_embeddings, entity_all_embeddings



NameError: name 'nn' is not defined

## Loss

In [5]:
def calculate_loss(anchor_emb, pos_emb, neg_emb, ent_emb, batch_size, reg):
    calc_reg_loss = EmbLoss()
    rec_loss = bpr_loss(anchor_emb, pos_emb, neg_emb)
    reg_loss = reg * calc_reg_loss(anchor_emb, pos_emb, neg_emb, ent_emb) / batch_size
    return rec_loss, reg_loss 

In [6]:
def predict(u, rec, user_emb, item_emb):
    user_id  = rec.data.get_user_id(u)
    score = torch.matmul(user_emb[user_id], item_emb.transpose(0, 1))
    return score.cpu().numpy()  

## Train

In [39]:
def train():
    lst_train_losses = []
    lst_rec_losses = []
    lst_reg_losses = []
    lst_performances = []

    for ep in range(maxEpoch):
        train_losses = []
        rec_losses = []
        reg_losses = []

        
        for n, batch in enumerate(next_batch_pairwise_kg(rec.data, batchSize)):
            user_idx, pos_idx, neg_idx, e_idx = batch
            train_model.train()
            user_emb, item_emb, entity_emb = train_model()

            anchor_emb = user_emb[user_idx]
            pos_emb = item_emb[pos_idx]
            neg_emb = item_emb[neg_idx]
            ent_emb = entity_emb[e_idx]

            rec_loss, reg_loss = calculate_loss(anchor_emb, pos_emb, neg_emb, ent_emb, batchSize, reg)
            batch_loss = rec_loss + reg_loss 
            
            train_losses.append(batch_loss.item())
            rec_losses.append(rec_loss.item())
            reg_losses.append(reg_loss.item())
            optimizer.zero_grad()
            batch_loss.backward()
            torch.nn.utils.clip_grad_norm_(train_model.parameters(), 4)
            optimizer.step()

        batch_train_loss = np.mean(train_losses)
        scheduler.step(batch_train_loss)
        
        train_loss = np.mean(train_losses)
        rec_loss = np.mean(rec_losses)
        reg_loss = np.mean(reg_losses)

        lst_train_losses.append([ep, train_loss])
        lst_rec_losses.append([ep,rec_loss])
        lst_reg_losses.append([ep, reg_loss])

        # Evaluation
        train_model.eval()
        with torch.no_grad():
            user_emb, item_emb, _ = train_model()
#         rec.fast_evaluation(ep, user_emb, item_emb)
            _, data_ep = rec.fast_evaluation(train_model, ep, user_emb, item_emb)
        lst_performances.append(data_ep)

    rec.save_loss(lst_train_losses, lst_rec_losses, lst_reg_losses)
    rec.save_perfomance_training(lst_performances)
    user_emb, item_emb = rec.best_user_emb, rec.best_item_emb

    return user_emb, item_emb

## Test

In [40]:
def test(rec, user_emb, item_emb):
    def process_bar(num, total):
        rate = float(num) / total
        ratenum = int(50 * rate)
        r = '\rProgress: [{}{}]{}%'.format('+' * ratenum, ' ' * (50 - ratenum), ratenum*2)
        sys.stdout.write(r)
        sys.stdout.flush()

    # predict
    rec_list = {}
    user_count = len(rec.data.test_set)
    for i, user in enumerate(rec.data.test_set):
        # s_find_candidates = time.time()
        candidates = predict(user, rec, user_emb, item_emb)
        # e_find_candidates = time.time()
        # print("Calculate candidates time: %f s" % (e_find_candidates - s_find_candidates))
        # predictedItems = denormalize(predictedItems, self.data.rScale[-1], self.data.rScale[0])
        rated_list, li = rec.data.user_rated(user)
        for item in rated_list:
            candidates[rec.data.item[item]] = -10e8

        # s_find_k_largest = time.time()
        ids, scores = find_k_largest(rec.max_N, candidates)
        # e_find_k_largest = time.time()
        # print("Find k largest candidates: %f s" % (e_find_k_largest - s_find_k_largest))
        item_names = [rec.data.id2item[iid] for iid in ids]
        rec_list[user] = list(zip(item_names, scores))
        if i % 1000 == 0:
            process_bar(i, user_count)
    process_bar(user_count, user_count)
    print('')
    rec.evaluate(rec_list)

## Main

In [48]:
model = 'HGNN'
if model not in ['HGNN', 'LightGCN']:
    print("No model found.")
config = ModelConf('./conf/' + model + '.conf')

dataset = 'lastfm'
batchSize = int(config['batch_size'])
# lRate = float(config['learnRate'])
# maxEpoch = int(config['num.max.epoch'])
# reg = float(config['reg.lambda'])
maxEpoch = 500
lRates = [0.01, 0.001, 0.0001]
lrDecays = [0.9, 0.8, 0.5]
regs = [0.01, 0.001, 0.0001]
hyperDims = [32, 64, 128]
inputDims = [8, 16, 32]
ps = [0.1, 0.4, 0.6]
dropRates = [0.1, 0.3, 0.5]
nLayers = [1, 2, 3]
nHeads = [1]
nSelfAtt = [1]

if dataset == 'lastfm':
    training_set = "./dataset/lastfm/train.txt"
    test_set = './dataset/lastfm/test.txt'
    knowledge_data='./dataset/lastfm/processed/lastfm.kg'

elif dataset == 'ml-1m':
    training_set = './dataset/ml-1m/train.txt'
    test_set = './dataset/ml-1m/test.txt'
    knowledge_data ='./dataset/ml-1m/processed/ml-1m.kg'
hyperparameters = [lRates, lrDecays, regs, hyperDims, inputDims, ps, dropRates, nLayers, nHeads, nSelfAtt]

In [49]:
for params in product(*hyperparameters):
    lr, lr_decay, reg, hyper_dim, input_dim, prob, drop_rate, n_layers, n_heads, n_self_att = params
    print(params)
    args = {
        'lr': lr,
        'lr_decay': lr_decay,
        'reg': reg,
        'hyper_dim': hyper_dim,
        'input_dim': input_dim,
        'p': prob,
        'drop_rate': drop_rate,
        'n_layers': n_layers,
        'input_dim': input_dim,
        'hyper_dim': hyper_dim,
        'n_heads': n_heads,
        'n_self_att': n_self_att
    }
    training_data = FileIO.load_data_set(training_set, config['model.type'])
    test_data = FileIO.load_data_set(test_set, config['model.type'])
    knowledge_set = FileIO.load_kg_data(knowledge_data)

    rec = GraphRecommender(config, training_data, test_data, knowledge_set, **args)
    train_model = Model(rec.config, rec.data, args).to(device)
    optimizer  = torch.optim.Adam(train_model.parameters(), lr=lr)
    scheduler = ReduceLROnPlateau(optimizer, 'min', factor=lr_decay, patience=5)
    user_emb, item_emb = train()
    test(rec, user_emb, item_emb)

(0.01, 0.9, 0.01, 32, 8, 0.1, 0.1, 1, 1, 1)
Loading the dataset lastfm ....
constructing knowledge graph ...
Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.061000 s
Measure time: 0.009056 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
Epoch: 1, Hit Ratio:0.11703  |  Precision:0.07282  |  Recall:0.11567  |  NDCG:0.13534
*Best Performance* 
Epoch: 1, Hit Ratio:0.11703  |  Precision:0.07282  |  Recall:0.11567  |  NDCG:0.13534
------------------------------------------------------------------------------------------------------------------------
Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.060744 s
Measure time: 0.009097 s
---------------------------------------------------------------------------------------------------

Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.062472 s
Measure time: 0.009431 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
Epoch: 14, Hit Ratio:0.16503  |  Precision:0.10269  |  Recall:0.16725  |  NDCG:0.18945
*Best Performance* 
Epoch: 6, Hit Ratio:0.16631  |  Precision:0.10349  |  Recall:0.16658  |  NDCG:0.18867
------------------------------------------------------------------------------------------------------------------------
Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.062918 s
Measure time: 0.009381 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
Epoc

Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.064120 s
Measure time: 0.009457 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
Epoch: 27, Hit Ratio:0.17102  |  Precision:0.10642  |  Recall:0.17344  |  NDCG:0.1942
*Best Performance* 
Epoch: 26, Hit Ratio:0.17471  |  Precision:0.10872  |  Recall:0.1758  |  NDCG:0.20029
------------------------------------------------------------------------------------------------------------------------
Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.068079 s
Measure time: 0.009734 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
Epoch

Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.064692 s
Measure time: 0.009610 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
Epoch: 40, Hit Ratio:0.1812  |  Precision:0.11276  |  Recall:0.18265  |  NDCG:0.20722
*Best Performance* 
Epoch: 31, Hit Ratio:0.1835  |  Precision:0.11418  |  Recall:0.18593  |  NDCG:0.2068
------------------------------------------------------------------------------------------------------------------------
Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.065559 s
Measure time: 0.009965 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
Epoch:

Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.064045 s
Measure time: 0.009746 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
Epoch: 53, Hit Ratio:0.18477  |  Precision:0.11498  |  Recall:0.18508  |  NDCG:0.21151
*Best Performance* 
Epoch: 53, Hit Ratio:0.18477  |  Precision:0.11498  |  Recall:0.18508  |  NDCG:0.21151
------------------------------------------------------------------------------------------------------------------------
Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.063609 s
Measure time: 0.009597 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
Epo

...

Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.884833 s
Measure time: 0.009690 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
Epoch: 445, Hit Ratio:0.22361  |  Precision:0.13914  |  Recall:0.22428  |  NDCG:0.25151
*Best Performance* 
Epoch: 362, Hit Ratio:0.22679  |  Precision:0.14113  |  Recall:0.22773  |  NDCG:0.25345
------------------------------------------------------------------------------------------------------------------------
Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 1.163630 s
Measure time: 0.009980 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
E

Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 1.179757 s
Measure time: 0.009636 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
Epoch: 458, Hit Ratio:0.22488  |  Precision:0.13994  |  Recall:0.22557  |  NDCG:0.25233
*Best Performance* 
Epoch: 362, Hit Ratio:0.22679  |  Precision:0.14113  |  Recall:0.22773  |  NDCG:0.25345
------------------------------------------------------------------------------------------------------------------------
Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.881995 s
Measure time: 0.009744 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
E

Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.884231 s
Measure time: 0.010104 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
Epoch: 471, Hit Ratio:0.22335  |  Precision:0.13899  |  Recall:0.22433  |  NDCG:0.25209
*Best Performance* 
Epoch: 362, Hit Ratio:0.22679  |  Precision:0.14113  |  Recall:0.22773  |  NDCG:0.25345
------------------------------------------------------------------------------------------------------------------------
Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 1.160476 s
Measure time: 0.009883 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
E

Evaluating the model...
Progress: [                                                  ]0%

KeyboardInterrupt: 